In [1]:
import json
import datetime
import pytz
import pandas as pd
from collections import defaultdict

In [2]:
# open the fitbit file as downloaded from Open Humans
with open('fitbit-data.json') as f:
    d = f.readlines()
    d = "".join(d)
    fitbit = json.loads(d)

In [3]:
# which keys do we export to the CSV, not all keys are valid for the export as done below. 
# this selection works
keys = ['sleep-awakenings',
        'heart',
        'tracker-minutes-very-active', 
        'tracker-minutes-lightly-active', 
        'sleep-minutes-after-wakeup', 
        'tracker-elevation', 
        'tracker-steps', 
        'tracker-distance', 
        'time-in-bed', 
        'minutes-to-sleep', 
        'tracker-minutes-sedentary', 
        'tracker-calories', 
        'tracker-floors', 
        'sleep-start-time', 
        'tracker-minutes-fairly-active', 
        'awake-minutes', 
        'sleep-minutes', 
        'sleep-efficiency']

In [7]:
# create an empty list that will store the dataframes for the individual data types
dataframes = []
for key in keys:
    timestamp = []
    value = []
    if key != 'heart':
        for datarange in fitbit[key].keys():
            internal_key = list(fitbit[key][datarange].keys())[0]
            for entry in fitbit[key][datarange][internal_key]:
                timestamp.append(entry['dateTime'])
                value.append(entry['value'])
    else:
        for datarange in fitbit[key].keys():
            for day in fitbit['heart'][datarange]['activities-heart']:
                if 'restingHeartRate' in day['value'].keys():
                    timestamp.append(day['dateTime'])
                    value.append(day['value']['restingHeartRate'])
                internal_key = 'restingHeartRate'
    df = pd.DataFrame(data = {'date': timestamp, 
                              internal_key: value})
    dataframes.append(df)

# Merge the individual dataframes into a larger one with all columns
df_merged = dataframes.pop(0)
for df in dataframes:
    df_merged = pd.merge(df_merged, df, on='date') 
df_merged.index = pd.to_datetime(df_merged['date'])

In [8]:
# Export this file to CSV
df_merged.to_csv('fitbit_with_heart.csv')

# use this file for the Rscript